***CONSUMER START***

In [ ]:
from kafka import KafkaConsumer
import pickle
import json

***PREPARING KAFKA FOR DATA INCOMING***

In [ ]:
bootstrap_servers = ['localhost:9092']
consumer = KafkaConsumer('temp_02', bootstrap_servers=bootstrap_servers, auto_offset_reset='earliest', group_id='my-group')

***PREDICTING***

In [ ]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

with open("explainer.pkl", "rb") as f:
    explainer = pickle.load(f)

def predict(incomingData):
    data_dict = json.loads(incomingData)
    ID=data_dict['ID']
    del data_dict['ID']
    data_list=[value for value in data_dict.values()]
    y_pred_test=model.predict([data_list])
    return y_pred_test, ID

***DATA INCOMING***

In [ ]:
for msg in consumer:
    msg_i=msg.value.decode('utf-8')
    rL=predict(msg_i)
    
    if(rL[0]==1):
        print(f"The transaction with ID: {rL[1]} is Fraudulent")
        print(f"Here are its details:\n{msg_i}")

***CONSUMER STOP***

In [ ]:
consumer.close()
print("Consumer stopped")